Inform Python we will use the StatsModels Library

In [33]:
import statsmodels.api as st
import statsmodels.formula.api as smf

Use IRIS Dataset

In [34]:
# import the dataset, declare target and features
iris = st.datasets.get_rdataset('iris','datasets')
y = iris.data.Species 
X = iris.data.ix[:, 0:4]

In [35]:
#examine y
y

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
5         setosa
6         setosa
7         setosa
8         setosa
9         setosa
10        setosa
11        setosa
12        setosa
13        setosa
14        setosa
15        setosa
16        setosa
17        setosa
18        setosa
19        setosa
20        setosa
21        setosa
22        setosa
23        setosa
24        setosa
25        setosa
26        setosa
27        setosa
28        setosa
29        setosa
         ...    
120    virginica
121    virginica
122    virginica
123    virginica
124    virginica
125    virginica
126    virginica
127    virginica
128    virginica
129    virginica
130    virginica
131    virginica
132    virginica
133    virginica
134    virginica
135    virginica
136    virginica
137    virginica
138    virginica
139    virginica
140    virginica
141    virginica
142    virginica
143    virginica
144    virginica
145    virginica
146    virginica
147    virgini

Setup the Logistic Regression 

In [36]:
# add column of ones for our y-intercept (http://blog.yhat.com/posts/logistic-regression-and-python.html)
X['intercept'] = 1.0
#declare our model

MNlogit = st.MNLogit(y, X)


Perform the Regression using the Fit Method

In [37]:
result = MNlogit.fit(method = 'bfgs')
result

         Current function value: 0.057112
         Iterations: 35
         Function evaluations: 37
         Gradient evaluations: 37


//anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Print a summary of the results

In [13]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                Species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      140
Method:                           MLE   Df Model:                            8
Date:                Tue, 03 May 2016   Pseudo R-squ.:                  0.9480
Time:                        14:59:19   Log-Likelihood:                -8.5668
converged:                      False   LL-Null:                       -164.79
                                        LLR p-value:                 9.200e-63
=====================================================================================
Species=versicolor       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Sepal.Length          -1.4959    444.817     -0.003      0.997      -873.321   870.330
Sepal.Width           -8.0560    282.766     -0.028      0.977      -562.267   546.155
Petal.Length          11.9301    374.116      0.032      0.975      -721.324   745.184
Petal.Width            1.7039    759.366      0.002      0.998     -1486.627  1490.035
intercept              1.6444   1550.515      0.001      0.999     -3037.309  3040.597
--------------------------------------------------------------------------------------
Species=virginica       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Sepal.Length         -8.0348    444.835     -0.018      0.986      -879.896   863.827
Sepal.Width         -15.8195    282.793     -0.056      0.955      -570.083   538.444
Petal.Length         22.1797    374.155      0.059      0.953      -711.152   755.511
Petal.Width          14.0603    759.384      0.019      0.985     -1474.304  1502.425
intercept            -6.5053   1550.533     -0.004      0.997     -3045.494  3032.483
=====================================================================================
"""

## Initial discussion of the model

As we can see from the model, the Pseudo R-squared is high. However, there is one problem with this regression:

* The Z-tests on each parameter in the model output. All are so close to 1 and with such large standard errors that the probability that they are null can not be ignored.

This is even with a p-value on the log likelihood ratio indicating that the model as a whole could be used for prediction.

This brings us to...

## Tuning the model

If we suspect the model's fit, we should consider estimating the likelihood of a flower being a type of iris, but remove one of the exogenous variables. It is possible that at least one is highly correlated, causing an issue with the fit.

All else equal, I would approach it by taking out the highest p-value variable first. This is petal_width.

### Addendum

Also, for what it is worth, the documentation here specifies needing to specify an intercept: http://statsmodels.sourceforge.net/devel/_modules/statsmodels/discrete/discrete_model.html#MNLogit

The broader question is what specifying the intercept allows the model to do.

In [51]:
#So let's consider tuning the model by taking out petal_width.

X_new = iris.data.ix[:, 0:3]

#Add constant

X_new = st.add_constant(X_new, prepend = False)

#Declar the model
MNlogit2 = st.MNLogit(y, X_new)

In [53]:
#Fitted the model and printed summary stats
result2 = MNlogit2.fit(method = 'bfgs')
result2.summary()

         Current function value: 0.088094
         Iterations: 35
         Function evaluations: 38
         Gradient evaluations: 38


//anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                Species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      142
Method:                           MLE   Df Model:                            6
Date:                Tue, 03 May 2016   Pseudo R-squ.:                  0.9198
Time:                        22:59:58   Log-Likelihood:                -13.214
converged:                      False   LL-Null:                       -164.79
                                        LLR p-value:                 1.724e-62
=====================================================================================
Species=versicolor       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Sepal.Length          -2.9862   2118.657     -0.001      0.999     -4155.477  4149.505
Sepal.Width          -11.4595   1276.495     -0.009      0.993     -2513.344  2490.425
Petal.Length          14.4379    717.141      0.020      0.984     -1391.133  1420.009
const                 10.8274   7182.336      0.002      0.999     -1.41e+04  1.41e+04
--------------------------------------------------------------------------------------
Species=virginica       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Sepal.Length         -5.7800   2118.657     -0.003      0.998     -4158.272  4146.712
Sepal.Width         -13.9999   1276.497     -0.011      0.991     -2515.888  2487.888
Petal.Length         24.8938    717.146      0.035      0.972     -1380.687  1430.475
const               -15.9440   7182.342     -0.002      0.998     -1.41e+04  1.41e+04
=====================================================================================
"""

### Tuning the model continued.

Dropping Petal Width did not improve the model. The p-values for the remaining variables are still too high. Testing dropping sepal length.

In [56]:
#Trimmed data for model tuning. No petal width or sepal length
X_2 = iris.data.ix[:, 1:3]

#Adding constant
X_2 = st.add_constant(X_2, prepend = False)

#Declaring model
MNlogit3 = st.MNLogit(y, X_2)

In [58]:
#Fit the model and results
#Note that I did not get an Nan back, so no need to switch to a model like bfgs.
results3 = MNlogit3.fit()

         Current function value: 0.105039
         Iterations: 35


//anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [59]:
results3.summary()
#Not good...

<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                Species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      144
Method:                           MLE   Df Model:                            4
Date:                Tue, 03 May 2016   Pseudo R-squ.:                  0.9044
Time:                        23:04:28   Log-Likelihood:                -15.756
converged:                      False   LL-Null:                       -164.79
                                        LLR p-value:                 2.823e-63
=====================================================================================
Species=versicolor       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Sepal.Width          -24.2311      9e+06  -2.69e-06      1.000     -1.76e+07  1.76e+07
Petal.Length          37.1804    2.9e+06   1.28e-05      1.000     -5.69e+06  5.69e+06
const                -20.3505   2.27e+07  -8.98e-07      1.000     -4.44e+07  4.44e+07
--------------------------------------------------------------------------------------
Species=virginica       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Sepal.Width         -26.7403      9e+06  -2.97e-06      1.000     -1.76e+07  1.76e+07
Petal.Length         46.2787    2.9e+06   1.59e-05      1.000     -5.69e+06  5.69e+06
const               -57.4087   2.27e+07  -2.53e-06      1.000     -4.44e+07  4.44e+07
=====================================================================================
"""

In [60]:
#One more go on tuning. Just Sepal Width.
X_3 = iris.data.ix[:, 1:2]

#Add intercept

X_3 = st.add_constant(X_3, prepend = False)

#Declare model

MNlogit4 = st.MNLogit(y, X_3)

#results
results4 = MNlogit4.fit()
#Note, done without a warning on iterations!

Optimization terminated successfully.
         Current function value: 0.841790
         Iterations 7


In [61]:
results4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                Species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      146
Method:                           MLE   Df Model:                            2
Date:                Tue, 03 May 2016   Pseudo R-squ.:                  0.2338
Time:                        23:07:05   Log-Likelihood:                -126.27
converged:                       True   LL-Null:                       -164.79
                                        LLR p-value:                 1.860e-17
=====================================================================================
Species=versicolor       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Sepal.Width           -6.1190      0.991     -6.173      0.000        -8.062    -4.176
const                 18.8584      3.064      6.154      0.000        12.853    24.864
--------------------------------------------------------------------------------------
Species=virginica       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Sepal.Width          -4.0791      0.844     -4.836      0.000        -5.732    -2.426
const                12.9973      2.688      4.835      0.000         7.728    18.266
=====================================================================================
"""

In [68]:
#Exponentiating the parameters
#Versicolor
print "The odds ratio for versicolor for a unit change in Sepal Width is:", np.exp(results4.params[0][0])
#Virginica
print "The odds ratio for virginica for a unit change in Sepal Width is:", np.exp(results4.params[1][0])

The odds ratio for versicolor for a unit change in Sepal Width is: 0.00220074015845
The odds ratio for virginica for a unit change in Sepal Width is: 0.0169227214054


### Tuning Discussion continued.

With only one column now, the p-value for it is low enough that I would reject the null that the coefficient is zero.

So what does the coefficient mean? It is not exponentiated into its odds ratio. However, above are the values exponentiated into odds ratios. Essentially, the odds increase by 1.6% as sepal width increases by one unit of it being a versicolor iris.

Before I move on, I would like to see what regressing on single variables other than sepal width does, and which has the "better" fit.

In [80]:
#Sepal Length
X_length = iris.data.ix[:, 0:1]

#Add constant
X_length = st.add_constant(X_length, prepend = False)

#Model
MNLogit_length = st.MNLogit(y, X_length)

#Results
results_length = MNLogit_length.fit()

Optimization terminated successfully.
         Current function value: 0.606893
         Iterations 8


In [85]:
#Summary of regressing on Sepal Length
results_length.summary()
#Note the pseudo R squared of .4476

<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                Species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      146
Method:                           MLE   Df Model:                            2
Date:                Tue, 03 May 2016   Pseudo R-squ.:                  0.4476
Time:                        23:21:44   Log-Likelihood:                -91.034
converged:                       True   LL-Null:                       -164.79
                                        LLR p-value:                 9.276e-33
=====================================================================================
Species=versicolor       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Sepal.Length           4.8157      0.907      5.310      0.000         3.038     6.593
const                -26.0819      4.889     -5.335      0.000       -35.665   -16.499
--------------------------------------------------------------------------------------
Species=virginica       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Sepal.Length          6.8464      1.022      6.698      0.000         4.843     8.850
const               -38.7590      5.691     -6.811      0.000       -49.913   -27.605
=====================================================================================
"""

In [93]:
#Petal Width
X_petal_width = iris.data.ix[:, 3]

#Add constant
X_petal_width = st.add_constant(X_petal_width, prepend = False)

#Model
MNLogit_petal_width = st.MNLogit(y, X_petal_width)

#Results
results_petal_width = MNLogit_petal_width.fit()
results_petal_width.summary()
#High R-squared, but high p-value for variable.

         Current function value: 0.111403
         Iterations: 35


//anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                Species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      146
Method:                           MLE   Df Model:                            2
Date:                Tue, 03 May 2016   Pseudo R-squ.:                  0.8986
Time:                        23:25:43   Log-Likelihood:                -16.710
converged:                      False   LL-Null:                       -164.79
                                        LLR p-value:                 4.887e-65
=====================================================================================
Species=versicolor       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Petal.Width          101.2540   2.12e+05      0.000      1.000     -4.14e+05  4.15e+05
const                -83.4444   2.11e+05     -0.000      1.000     -4.14e+05  4.14e+05
--------------------------------------------------------------------------------------
Species=virginica       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Petal.Width         114.2015   2.12e+05      0.001      1.000     -4.14e+05  4.15e+05
const              -104.5700   2.11e+05     -0.000      1.000     -4.15e+05  4.14e+05
=====================================================================================
"""

In [92]:
#Petal Length
X_petal_length = iris.data.ix[:, 2]

#Add constant
X_petal_length = st.add_constant(X_petal_length, prepend = False)

#Model
MNLogit_petal_length = st.MNLogit(y, X_petal_length)

#Results
results_petal_length = MNLogit_petal_length.fit()
results_petal_length.summary()
#High R-squared, but high p-value for variable.

         Current function value: 0.111440
         Iterations: 35


//anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                Species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      146
Method:                           MLE   Df Model:                            2
Date:                Tue, 03 May 2016   Pseudo R-squ.:                  0.8986
Time:                        23:24:50   Log-Likelihood:                -16.716
converged:                      False   LL-Null:                       -164.79
                                        LLR p-value:                 4.914e-65
=====================================================================================
Species=versicolor       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Petal.Length          34.3958   1.39e+04      0.002      0.998     -2.73e+04  2.74e+04
const                -84.4463   3.37e+04     -0.003      0.998     -6.62e+04   6.6e+04
--------------------------------------------------------------------------------------
Species=virginica       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Petal.Length         43.3978   1.39e+04      0.003      0.998     -2.73e+04  2.74e+04
const              -128.2272   3.37e+04     -0.004      0.997     -6.62e+04   6.6e+04
=====================================================================================
"""

In [95]:
#What do sepal length and width do together?
#Sepals
X_sepals = iris.data.ix[:, 0:2]

#Add constant
X_sepals = st.add_constant(X_sepals, prepend = False)

#Model
MNLogit_sepals = st.MNLogit(y, X_sepals)

#Results
results_sepals = MNLogit_sepals.fit(method = 'bfgs')
results_sepals.summary()
#High R-squared, but constant irrelevant. Drop it?

         Current function value: 0.413469
         Iterations: 35
         Function evaluations: 38
         Gradient evaluations: 38


//anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                Species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      144
Method:                           MLE   Df Model:                            4
Date:                Tue, 03 May 2016   Pseudo R-squ.:                  0.6236
Time:                        23:27:12   Log-Likelihood:                -62.020
converged:                      False   LL-Null:                       -164.79
                                        LLR p-value:                 2.415e-43
=====================================================================================
Species=versicolor       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Sepal.Length          10.0280      4.619      2.171      0.030         0.975    19.081
Sepal.Width          -18.3559      9.396     -1.954      0.051       -36.772     0.060
const                  2.1349     16.989      0.126      0.900       -31.162    35.432
--------------------------------------------------------------------------------------
Species=virginica       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Sepal.Length         11.3676      4.639      2.451      0.014         2.276    20.460
Sepal.Width         -17.7987      9.407     -1.892      0.058       -36.237     0.639
const                -7.8037     17.086     -0.457      0.648       -41.292    25.685
=====================================================================================
"""

In [96]:
#What do sepal length and width do together?
#Sepals
X_sepals2 = iris.data.ix[:, 0:2]



#Model
MNLogit_sepals2 = st.MNLogit(y, X_sepals2)

#Results
results_sepals2 = MNLogit_sepals2.fit()
results_sepals2.summary()
#High R-squared, but constant irrelevant. Drop it?

Optimization terminated successfully.
         Current function value: 0.490662
         Iterations 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                Species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      146
Method:                           MLE   Df Model:                            2
Date:                Tue, 03 May 2016   Pseudo R-squ.:                  0.5534
Time:                        23:28:21   Log-Likelihood:                -73.599
converged:                       True   LL-Null:                       -164.79
                                        LLR p-value:                 2.486e-40
=====================================================================================
Species=versicolor       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
Sepal.Length           7.3240      2.261      3.240      0.001         2.893    11.755
Sepal.Width          -12.9117      3.999     -3.229      0.001       -20.749    -5.074
--------------------------------------------------------------------------------------
Species=virginica       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Sepal.Length          7.8319      2.274      3.444      0.001         3.375    12.288
Sepal.Width         -13.9867      4.032     -3.469      0.001       -21.889    -6.085
=====================================================================================
"""

## A Conclusion

After looking at the model output and tuning it, it was clear that:

1) Petal length and width did not add predictive power.

2) Sepal Length or Sepal Width seem to have predictive power and be non-zero in impact.

3) Sepal Length and Sepal Width together seem to have a better model fit than either Sepal Length or Width on their own. Interestingly, on their own, both need an intercept. However, together, the intercept is viewed as highly likely to be zero (can not reject null).

The relevant pseudo R-Squareds are:

1) Sepal Length and Sepal Width, no constant: 0.5534

2) Sepal Length, with constant: 0.4476

3) Sepal Width, with constant: 0.2338

Finally, while the coefficients are interesting, they are all in log odds format. To convert to an odds ration, exponentiate the function. And then if you want the probablity, set the result equal to p/(1-p) where p is the probability.

N.B. So Multinomial Logistic regression produces an output of k-1 models. For this model, k was 3 for 3 levels for our y, the type of iris. Hence, we see two models above.